In [ ]:
#!/usr/bin/env python
# coding: utf-8

import os
import sys
import time

In [ ]:
import torch
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sys.path.insert(1, os.path.join("C:\\", "Soham", "Git", "spectral-NN", "source_codes"))
#sys.path.insert(1, os.path.join("C:\\", "Users", "Soham", "Git", "spectral-NN", "source_codes"))
#sys.path.insert(1, os.path.join("/home", "soham", "GitHub", "spectral-NN", "source_codes"))

import SpectralNetworks as spectNN
import Important_functions as Ifn
#import Other_functions as Ofn

In [ ]:
dirc = "C:\\Soham\\Git\\spectral-NN\\Data\\"
#dirc = "C:\\Users\\Soham\\Git\\spectral-NN\\Data\\"
#dirc = "/home/soham/GitHub/spectral-NN/Data/"
repl = 0

In [ ]:
    print('Example'+str(repl+1)+':')
    file = dirc+'locations'+str(repl+1)+'.dat'
    u = np.loadtxt(dirc+"locations.dat",dtype="float32")
    if len(u.shape)==1:
        D, d = len(u), 1
        u = u.reshape(D,1)
    else:
        D, d = u.shape
    u = torch.from_numpy(u)
    file = dirc+'Example'+str(repl+1)+'.dat'
    x = np.loadtxt(file,dtype='float32')
    N = x.shape[0]
    if x.shape[1] != D:
        exit('Data shape mismatch!! Aborting..')
    print('N='+str(N)+', D='+str(D)+', d='+str(d))

    x = torch.from_numpy(x)
    x = x - torch.mean(x,dim=0,keepdim=True)

In [ ]:
M, L, depth, width = 10, 10, 3, 20
act_fn=torch.nn.Sigmoid()
init=torch.nn.init.xavier_normal_

#model = spectNN.spectralNNShallow(N,d,M,L,act_fn,init)
#model = spectNN.spectralNNDeep(N,d,M,L,depth,width,act_fn,init)
model = spectNN.spectralNNDeepshared1(N,d,M,L,depth,width,act_fn,init)
#model = spectNN.spectralNNDeepshared2(N,d,M,L,depth,width,act_fn,init)
#model = spectNN.spectralNNDeepshared3(N,d,M,L,depth,width,act_fn,init)

#wt_fn = Ifn.kern_truncated
#wt_fn = Ifn.kern_bartlett
wt_fn = Ifn.kern_parzen
loss = Ifn.loss_spectralNN(N, wt_fn, grid_size=100, q=10)
optimizer = torch.optim.Adam(model.params,lr=0.01)
epochs = 1000

In [ ]:
print("Fitting the model ...")
start_time = time.time()
l_tr = []
for epoch in range(epochs):
    l = loss.loss_fn(x,model(u))
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    l_tr.append(l.item())
time_ellapsed = time.time() - start_time
print("Time taken: {} seconds" .format(time_ellapsed))

fig, ax = plt.subplots(figsize=(5,5))
ax.plot(l_tr)
plt.show()

with torch.no_grad():
    num = loss.loss_fn(x,model(u)).item()
    den = loss.loss_fn(x,0*x).item()
    print("Relative error: {:.2f}%" .format(num/den*100))
    print("Numerator: {:.4f}, Denominator: {:.4f}" .format(num,den))

In [ ]:
import Important_functions as Ifn

In [ ]:
spect_dens_est = Ifn.spectral_density_evaluation(model, q=10, wt_fn=wt_fn)

In [ ]:
v = u
theta = (2*torch.rand(1)-1)*np.pi
f_hat = spect_dens_est.evaluate(theta, u, v)
print(f_hat)